In [ ]:
!pip install mxnet
!pip install gluonnlp
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.3 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 14.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl size=619647 sha256=05b563fd908c25dfd2ca369091e3ae9137a47aaaafd3694ca3ab4fe52f0c5165
  Stored in directory: /root/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: http

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

device = torch.device("cuda:0")

torch >= 1.8.1
transformers >= 4.8.2
sentencepiece >= 0.1.91



In [ ]:
# 학습용 데이터셋 불러오기
import pandas as pd
dataset_train1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2차프로젝트/data/1dataset.csv')
dataset_train1.head()
# 라벨링
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(dataset_train1['y'])
dataset_train1['y'] = encoder.transform(dataset_train1['y'])
dataset_train1.head()

mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: 'domestic',
 1: 'export',
 2: 'finance',
 3: 'management',
 4: 'people',
 5: 'skill',
 6: 'startup'}

In [ ]:
# Train / Test set 분리
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset_train1, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))

train shape is: 614
test shape is: 154


In [ ]:
train

,X,y
60,광주광역시 소재 기업의 안정적인 경영을 도모하기 위하여 경영안정자금을 융자 지원해 ...,2
618,광주 지역 내 우수기업 보유 기업을 발굴하고 투자유치 역량강화 지원을 통해 기업의 ...,3
346,코로나19로 인한 소비 유통환경 변화에 대응하여 부산소재의 수산(물)식품 소비촉진 ...,0
294,광주광역시 위기산업(가전 및 자동차)의 실직(예정)자/퇴직자의 구직비용절감 및 면접...,4
231,부산 지역 IT산업 중소기업의 인력난 해소 및 청년일자리 창출을 도모하고자 IT산업...,4
...,...,...
71,충남지역 기업의 일시적 자금 유동성 해소를 통한 제조업 경영 안정화를 위해 운전자금...,2
106,미래차 산업 생태계를 조성하고 도내 자율 주행 산업의 육성을 위해 충북 소재 중소ㆍ...,5
270,"경북 영천시 내 친환경 모빌리티 분야 전문인력 발굴ㆍ육성, 신규 일자리창출을 통한 ...",4
435,대전 소재 중소기업을 대상으로 해외 분쟁대응 경쟁력 강화를 위하여 지식재산권 보호 ...,1


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# 파라미터 설정
max_len = 64   # 텍스트 데이터 최대 길이
batch_size = 64
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-8vn0_j28/kobert-tokenizer_2ee44e6cd5b74450ba7e6f83e4b22962
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-8vn0_j28/kobert-tokenizer_2ee44e6cd5b74450ba7e6f83e4b22962
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4648 sha256=9295ecf301ba2d46f5a69a2495247830fe59917c718560d9dd28ca81ddcb9540
  Stored in directory: /tmp/pip-ephem-wheel-cache-95ugrv12/wheels/f7/cb/29/1a737fe71e5108dc30b04ea4a990f78ed271fa537aaf3fce7c
Successfully built kobert-tokenizer


In [ ]:
from gluonnlp.data import SentencepieceTokenizer
from kobert import get_tokenizer
tok_path = get_tokenizer()
sp  = SentencepieceTokenizer(tok_path)

In [ ]:
import torch
from kobert import get_pytorch_kobert_model

In [ ]:
input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
model, vocab  = get_pytorch_kobert_model()
sequence_output, pooled_output = model(input_ids, input_mask, token_type_ids)
pooled_output.shape
torch.Size([2, 768])

# Last Encoding Layer
sequence_output[0]

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


tensor([[-0.2461,  0.2428,  0.2590,  ..., -0.4861, -0.0731,  0.0756],
        [-0.2478,  0.2420,  0.2552,  ..., -0.4877, -0.0727,  0.0754],
        [-0.2472,  0.2420,  0.2561,  ..., -0.4874, -0.0733,  0.0765]],
       grad_fn=<SelectBackward0>)

In [ ]:
tokenizer = SentencepieceTokenizer(get_tokenizer())
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
model.eval()
# model.train()

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2차프로젝트/data/1dataset.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(data['y'])
data['y'] = encoder.transform(data['y'])
data.head()

mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: 'domestic',
 1: 'export',
 2: 'finance',
 3: 'management',
 4: 'people',
 5: 'skill',
 6: 'startup'}

In [ ]:
data_list = []
for ques, label in zip(data['X'], data['y'])  :
    data = []   
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [ ]:
data_list

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5  
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [ ]:
tokenizer = get_tokenizer()

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
transform = nlp.data.BERTSentenceTransform(
            sp, max_seq_length=max_len,vocab=vocab, pad=64, pair=False)

In [ ]:
train_sentences = [transform([i[0]]) for i in dataset_train]

In [ ]:
train_labels = [np.int32(i[1]) for i in dataset_train]

In [ ]:
train_sentences

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[0]]) for i in dataset]
        self.labels = [np.int32(i[1]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
data_train[0]

(array([   2, 4151, 5577, 7095, 2427, 3463, 7788, 2931, 5464, 7088, 2361,
         495, 7944, 6516, 7788, 7147, 2931, 5464, 1130, 7088, 3457, 7828,
        4590, 4329, 4263,  495, 7295, 5414, 4151, 5577, 7088, 1652, 4316,
        6443, 4331, 7848, 1788, 6124, 2609, 7139,  517,   54,    3,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(43, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 3)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=7)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=7)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 7 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   #클래스 수 조정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(19):    #epoch
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

<ipython-input-65-4ed96e134b4b>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/10 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.027701523154973984 train acc 1.0
epoch 1 train acc 0.996875


<ipython-input-65-4ed96e134b4b>:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/3 [00:00<?, ?it/s]

...

epoch 18 test acc 0.8004807692307692


  0%|          | 0/10 [00:00<?, ?it/s]

epoch 19 batch id 1 loss 0.007516505196690559 train acc 1.0
epoch 19 train acc 1.0


  0%|          | 0/3 [00:00<?, ?it/s]

epoch 19 test acc 0.8004807692307692


In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=7)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("내수")
            elif np.argmax(logits) == 1:
                test_eval.append("수출")
            elif np.argmax(logits) == 2:
                test_eval.append("금융")
            elif np.argmax(logits) == 3:
                test_eval.append("경영")
            elif np.argmax(logits) == 4:
                test_eval.append("인력")
            elif np.argmax(logits) == 5:
                test_eval.append("기술")
            elif np.argmax(logits) == 6:
                test_eval.append("창업")

        print(">> 이 지원사업은 " + test_eval[0] + " 에 대한 사업입니다.")

In [ ]:
# 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("입력 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

입력 : 농림축산식품 R&D를 통해 개발된 기술의 사장을 방지하고 산업화 단계로 진입할 수 있도록 공공R&D사업화 및 민간R&D사업화를 지원해 드리는 사업입니다.
>> 이 지원사업은 기술 에 대한 사업입니다.


입력 : 수출
>> 이 지원사업은 경영 에 대한 사업입니다.


입력 : 독일로 수출 지원
>> 이 지원사업은 경영 에 대한 사업입니다.


입력 : 청년 일자리 제공
>> 이 지원사업은 경영 에 대한 사업입니다.


입력 : 채용
>> 이 지원사업은 경영 에 대한 사업입니다.


입력 : 지역 청년에게 적합한 지역 일자리를 발굴ㆍ제공하여 청년들의 지역안착을 위해 청년을 정규직으로 신규 채용할 계획이 있는 사회적경제기업을 대상으로 인건비를 지원해 드리는 사업입니다.
>> 이 지원사업은 인력 에 대한 사업입니다.


입력 : 우수한 기술력을 보유한 창업기업의 기술력 향상을 위하여 서울시가 지원하는 2022년도 기술이전 지원사업을 아래와 같이 공고 합니다.
>> 이 지원사업은 기술 에 대한 사업입니다.


입력 : 선문대학교 창업지원센터에서는 4차 산업혁명 분야의 우수한 기술과 창업아이템을 보유한 예비창업자를 육성하기 위한 2022년 과학벨트 창업성장지원사업에 참여할 예비창업자를 다음과 같이 모집합니다.
>> 이 지원사업은 창업 에 대한 사업입니다.


입력 : 0
